In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
%matplotlib inline

2023-09-14 13:57:18.581736: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-14 13:57:18.607622: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-14 13:57:18.608094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 13:57:19.108234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tensorflow import keras
from tensorflow.keras import models, layers, callbacks, activations, optimizers

In [3]:
def data_arrays(data_path):
    dataset = h5py.File(data_path, 'r')
    x_train = np.array(dataset['X_train']).transpose([0,2,1])
    y_train = np.array(dataset['Y_train'])
    x_valid = np.array(dataset['X_valid']).transpose([0,2,1])
    y_valid = np.array(dataset['Y_valid'])
    x_test = np.array(dataset['X_test']).transpose([0,2,1])
    y_test = np.array(dataset['Y_test'])
    alphabet = 'ACGT'

    
    return x_train, y_train, x_valid, y_valid, x_test, y_test, alphabet


In [ ]:
x_train, y_train, x_valid, y_valid, x_test, y_test, alphabet = data_arrays(data_path= '/home/pgarcia/rbp_project/data/HNRNPK_K562_200.h5')
print(x_train.shape, y_train.shape)

In [5]:
def input_shape_check(x_train, x_test,x_valid):
    if x_train.shape[-1] == 4:
        print("Input shape is correct: " + x_train.shape)
    else:
        # Adjust input shape
        x_train = x_train[:,:,:4]
        x_test = x_test[:,:,:4]
        x_valid = x_valid[:,:,:4]

        # Print the shape of the training data
        print("Input shape adjusted:")
        print(x_train.shape)
        print(x_test.shape)
        print(x_valid.shape)
    return x_train, x_test, x_valid

In [6]:
x_train, x_test, x_valid = input_shape_check(x_train, x_test,x_valid)

Input shape adjusted:
(11328, 200, 4)
(3237, 200, 4)
(1619, 200, 4)


In [7]:
x_train.shape

(11328, 200, 4)

In [ ]:
y_test#checks whether classification or regression task

# DeepBind Model

In [ ]:
def deepbind_test():
    #Build the model
    model = models.Sequential()
    #layer1
    model.add(layers.InputLayer(input_shape=(200, 4))) # 4 channel input
    #layer2
    model.add(layers.Conv1D(filters=16, kernel_size=24, padding='same'))
    model.add(layers.Activation(activations.relu))
    #layer3
    model.add(layers.MaxPooling1D(pool_size=25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(units=32, activation='relu')) #model says "one hidden layer with 32 ReLu units"?
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(units=1, activation='sigmoid'))
    
    
    #print(model.summary())
    
    
    return model

In [ ]:
model = deepbind_test()
loss = keras.losses.BinaryCrossentropy()
opti = tf.keras.optimizers.Adam(learning_rate = 0.005)

# Compile the model
model.compile(
    loss=loss,
    optimizer=opti, 
    metrics=[
        tf.keras.metrics.AUC(curve="ROC", name="auroc"),
        tf.keras.metrics.AUC(curve="PR", name="aupr")  # add AUPR curve to track dataset bias
    ]
    )

# Define an early stopping callback
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Train the model
model.fit(
    x_train, y_train, 
    batch_size=100, 
    epochs=100, 
    validation_data=(x_valid, y_valid),
    callbacks=[es_callback]
)

In [ ]:
#evaluate the model
model.evaluate(x_test, y_test)

# DeepBind Exp in first Conv layer

In [ ]:
def deepbind_exp_test():
    #Build the model
    model = models.Sequential()
    #layer1
    model.add(layers.InputLayer(input_shape=(200, 4))) # 4 channel input
    #layer2
    model.add(layers.Conv1D(filters=16, kernel_size=24, padding='same'))
    model.add(layers.Activation(activations.exponential))
    #layer3
    model.add(layers.MaxPooling1D(pool_size=25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(units=32, activation='relu')) #model says "one hidden layer with 32 ReLu units"?
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(units=1, activation='sigmoid'))
    
    
    #print(model.summary())
    
    
    return model

In [ ]:
model = deepbind_exp_test()
loss = keras.losses.BinaryCrossentropy()
opti = tf.keras.optimizers.Adam(learning_rate = 0.005)

# Compile the model
model.compile(
    loss=loss,
    optimizer=opti, 
    metrics=[
        tf.keras.metrics.AUC(curve="ROC", name="auroc"),
        tf.keras.metrics.AUC(curve="PR", name="aupr")  # add AUPR curve to track dataset bias
    ]
    )

# Define an early stopping callback
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Train the model
model.fit(
    x_train, y_train, 
    batch_size=100, 
    epochs=100, 
    validation_data=(x_valid, y_valid),
    callbacks=[es_callback]
)

In [ ]:
#evaluate the model
model.evaluate(x_test, y_test)

# Saliency Maps

In [ ]:
#get model predictions for test sequences
predictions = model.predict(x_test)

# Get the top num_plots predictions
num_plots = 10

# Get the sorted indices
sorted_indices = np.argsort(predictions[:, 0])[::-1]

# Extract the top num_plots sequences
X = x_test[sorted_indices[:num_plots]]

# Reshape X to (num_plots, 200, 4)
X = X.reshape((num_plots, 200, 4))




#########################################################
import tensorflow as tf

@tf.function
def calculate_saliency_map(X, model, class_index=0):
  """fast function to generate saliency maps"""
  if not tf.is_tensor(X):
    X = tf.Variable(X)

  with tf.GradientTape() as tape:
    tape.watch(X)
    output = model(X)[:,class_index]
  return tape.gradient(output, X)

saliency_map = calculate_saliency_map(X, model)
saliency_map = saliency_map.numpy()

#########################################################
import pandas as pd
import logomaker

def plot_saliency_map(scores, alphabet, ax=None):
  L,A = scores.shape
  counts_df = pd.DataFrame(data=0.0, columns=list(alphabet), index=list(range(L)))
  for a in range(A):
    for l in range(L):
      counts_df.iloc[l,a] = scores[l,a]

  if not ax:
    ax = plt.subplot(1,1,1)
  logomaker.Logo(counts_df, ax=ax)


saliency_scores = saliency_map * X
for scores in saliency_scores:
  fig = plt.figure(figsize=(20,1))
  ax = plt.subplot(1,1,1)
  plot_saliency_map(scores, alphabet, ax)

# Save model data to file

In [ ]:
# os.makedirs('/home/pgarcia/rbp_project/models_deepbind')# Creating a directory
# model.save('models_deepbind/HNRNPK_K562_200.h5')   # Saving model

# Representation Learning Model

In [ ]:
def repre_test():
    
    #Build the model
    tf.keras.backend.clear_session()
    model = models.Sequential()
    # layer1
    model.add(layers.InputLayer(input_shape=(200, 4))) # 4 channel input
    
    # layer2
    l2_regularizer = tf.keras.regularizers.L2(1e-6)
    model.add(layers.Conv1D(filters=30, kernel_size=19, strides=1, padding='same', kernel_regularizer=l2_regularizer))
    # add batch normalization
    # batch_norm = tf.keras.layers.BatchNormalization()
    model.add(layers.Activation(activations.relu))

    # layer3
    model.add(layers.MaxPooling1D(pool_size=2, strides=2))
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv1D(filters=30, kernel_size=19, strides=1, padding='same'))
    # model.add(layers.Conv1D(filters=30, kernel_size=19, strides= 1, padding='same', kernel_regularizer=l2_regularizer))
    model.add(layers.MaxPooling1D(pool_size=50, strides=50))
    model.add(layers.Dropout(0.1))
    model.add(layers.Flatten())
    
    model.add(layers.Dense(units=512, activation='relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(units=1, activation='sigmoid'))

    # model.summary()

    
    return model

In [ ]:
#Compile the model
model = repre_test()
loss = keras.losses.BinaryCrossentropy()
opti = tf.keras.optimizers.Adam(learning_rate=0.003)
model.compile(
    loss=loss, 
    optimizer=opti, 
    metrics=[
        tf.keras.metrics.AUC(curve="ROC", name="auroc"),
        tf.keras.metrics.AUC(curve="PR", name="aupr")
    ]
    ) # add AUPR curve to track dataset bias

# Define an early stopping callback
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.fit(
    x_train, y_train, 
    batch_size=100, 
    epochs=100, 
    validation_data=(x_valid, y_valid),
    callbacks=[es_callback]
)

In [ ]:
#evaluate the model
model.evaluate(x_test, y_test)

# Representation Learning Model w/ Exp function in 1D Conv Layer

In [8]:
def repre_exp_test():
    
    #Build the model
    tf.keras.backend.clear_session()
    model = models.Sequential()
    # layer1
    model.add(layers.InputLayer(input_shape=(200, 4))) # 4 channel input
    
    # layer2
    l2_regularizer = tf.keras.regularizers.L2(1e-6)
    model.add(layers.Conv1D(filters=30, kernel_size=19, strides=1, padding='same', kernel_regularizer=l2_regularizer))
    # add batch normalization
    # batch_norm = tf.keras.layers.BatchNormalization()
    model.add(layers.Activation(activations.relu))

    # layer3
    model.add(layers.MaxPooling1D(pool_size=2, strides=2))
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv1D(filters=30, kernel_size=19, strides=1, padding='same'))
    # model.add(layers.Conv1D(filters=30, kernel_size=19, strides= 1, padding='same', kernel_regularizer=l2_regularizer))
    model.add(layers.MaxPooling1D(pool_size=50, strides=50))
    model.add(layers.Dropout(0.1))
    model.add(layers.Flatten())
    
    model.add(layers.Dense(units=512, activation='relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(units=1, activation='sigmoid'))

    # model.summary()

    
    return model

In [9]:
#Compile the model
model = repre_exp_test()
loss = keras.losses.BinaryCrossentropy()
opti = tf.keras.optimizers.Adam(learning_rate=0.003)
model.compile(
    loss=loss, 
    optimizer=opti, 
    metrics=[
        tf.keras.metrics.AUC(curve="ROC", name="auroc"),
        tf.keras.metrics.AUC(curve="PR", name="aupr")
    ]
    ) # add AUPR curve to track dataset bias

# Define an early stopping callback
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.fit(
    x_train, y_train, 
    batch_size=100, 
    epochs=100, 
    validation_data=(x_valid, y_valid),
    callbacks=[es_callback]
)

Epoch 1/100


2023-09-14 13:57:41.299747: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


114/114 [==============================] - 2s 8ms/step - loss: 0.2907 - auroc: 0.9414 - aupr: 0.9537 - val_loss: 0.2203 - val_auroc: 0.9603 - val_aupr: 0.9720
Epoch 2/100
114/114 [==============================] - 1s 6ms/step - loss: 0.2017 - auroc: 0.9691 - aupr: 0.9748 - val_loss: 0.2643 - val_auroc: 0.9671 - val_aupr: 0.9768
Epoch 3/100
114/114 [==============================] - 1s 6ms/step - loss: 0.1883 - auroc: 0.9733 - aupr: 0.9783 - val_loss: 0.2172 - val_auroc: 0.9704 - val_aupr: 0.9779
Epoch 4/100
114/114 [==============================] - 1s 6ms/step - loss: 0.1722 - auroc: 0.9782 - aupr: 0.9820 - val_loss: 0.2034 - val_auroc: 0.9707 - val_aupr: 0.9775
Epoch 5/100
114/114 [==============================] - 1s 6ms/step - loss: 0.1508 - auroc: 0.9838 - aupr: 0.9859 - val_loss: 0.1886 - val_auroc: 0.9733 - val_aupr: 0.9788
Epoch 6/100
114/114 [==============================] - 1s 6ms/step - loss: 0.1388 - auroc: 0.9864 - aupr: 0.9883 - val_loss: 0.2400 - val_auroc: 0.9700 - val

In [10]:
#evaluate the model
model.evaluate(x_test, y_test)

102/102 [==============================] - 0s 1ms/step - loss: 0.1849 - auroc: 0.9781 - aupr: 0.9817


[0.18492381274700165, 0.9780533909797668, 0.9816553592681885]

# Saliency Analysis